In [3]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### 데이터 분석 & 목적----------------

- 데이터 구성 => 방법? 지도 /비지도
    * 데이터/피쳐/특성/변수/속성/독립변수/관측값--------결과/타겟/라벨/클래스/종속변수/반응변수

- 데이터 목적 => 결과/타겟/종속변수 ---> 수치형 : 회귀 , 범주형 : 분류

- 학습방법 => 해당하는 학습 방법 => all_estimator()

[1] 데이터 준비---

In [4]:
file='./data/student_info.csv'

In [5]:
std_df=pd.read_csv(file,usecols=[2,3])

In [6]:
std_df.head(3)

,weight_kg,height_cm
0,98,198
1,77,170
2,70,170


[2] 데이터 전처리---


- 데이터 정제 : 결측치, 중복값, 이상치, 모순값

- 데이터 표준화 : 단위, 자료형

- 데이터 정규화 : 수치값 => 값의 범위 : 통일 --> 스케일러

    * 범주형/글자 => ordinalEncoder, OneHotEncoder, LabelEncoder(y)
    
- 데이터 변형 : 특성 추가/생성

In [7]:
# 결측치 체크
std_df.isnull().sum()

weight_kg    0
height_cm    0
dtype: int64

In [8]:
bmi_value=round(std_df.weight_kg/((std_df.height_cm/100)*(std_df.height_cm/100)),1)

In [10]:
_bins=[0,18.5,23.0,25.0,30.0, max(bmi_value)]
bmi_level=pd.cut(bmi_value,bins=_bins,labels=[0,1,2,3,4])
bmi_level

0     2
1     3
2     2
3     1
4     2
     ..
85    3
86    1
87    1
88    3
89    4
Length: 90, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [11]:
bmi_level.value_counts()

1    28
3    19
2    18
4    17
0     8
dtype: int64

In [13]:
std_df['bmi_level']=bmi_level

In [15]:
std_df.head()

,weight_kg,height_cm,bmi_level
0,98,198,2
1,77,170,3
2,70,170,2
3,90,198,1
4,71,170,2


In [17]:
# 특성 데이터의 범위 확인
std_df.describe().T[['min','max']]

,min,max
weight_kg,43.0,110.0
height_cm,132.0,198.0


In [18]:
std_df.describe().T['max']-std_df.describe().T['min']

weight_kg    67.0
height_cm    66.0
dtype: float64

In [ ]:
# 수치형 특성 데이터 스케일링

st=StandardScaler()
st_data=st.fit_transform(std_df[std_df.columns[:-1]])

In [ ]:
mmt=MinMaxScaler()
mmt_data=mmt.fit_transform(std_df[std_df.columns[:-1]])

In [ ]:
# 데이터 -- 타켓/라벨/클래스 분리
data=std_df[std_df[:-1]]
target=std_df[std_df[-1]]

In [ ]:
data.shape, target.shape

[3]학습용 데이터 준비 ==> 학습용, 테스트용, 검증용---

- 분류 => 항목에 대한 비율
    * 데이터 추가
    * 업 샘플링/다운 샘플링
    * API의 매개변수에 비율조정한 매개변수 체크
- 교차검증

In [ ]:
# 타겟의 분류 데이터의 비율이 고르지 않음 => 비율에 맞도록 데이터 분리
X_train, X_test, y_train, y_test=train_test_split(data, target, random_state=11,test_size=0.2,stratify=target)



In [ ]:
len(y_train), y_train.value_counts()

In [ ]:
len(y_test), y_test.value_counts()

[4] 학습 모델 객체 생성 및 학습 진행---

In [19]:
# 분류 KNN 모델 객체 생성 --- n.neighbors=5 가 기본값
KnnModel=KNeighborsClassifier(n_neighbors=1)

In [20]:
for n in range(1,X_train.shape[0]):
    KnnModel.n_neighbors=n
    KnnModel.fit(X_train,y_train)
    sc=KnnModel.score(X_train,y_train)
    print(f'{n} : score : {sc}')
    scores.appens(sc)

NameError: name 'X_train' is not defined

In [ ]:
plt.plot(scores)
plt.xlabel('n_neighbors')
plt.ylabel('scores')

In [ ]:
# 결정된 n_neighbors=2 또는 7
KnnModel.n_neighbors = 7
KnnModel.fit(X_train,y_train)

[5] 예측 및 평가---

- 검증 데이터 => 예측 & 평가

- 테스트 데이터 => 에측 & 평가

In [21]:
y_pre=KnnModel.predict(X_test)    # 예측값

accuracy_score(y_test, y_pre)

NameError: name 'X_test' is not defined

In [ ]:
KnnModel.score(X_test, y_test)

[6] 결과---

- 성능 만족스럽지 못함 ==> 튜닝(하이퍼파라미터)

- 성능 만족스러움 => 모델 저장

In [22]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# 하이퍼 파라미터 => n_neighbors, p, weights
model=KNeighborsClassifier()
params={'n_neighbors' : [2,3,4,5], 'p':[1,2], 'weights':['uniform','distance']}


grid=GridSearchCV(model, param_grid=params, scoring="accuracy_score", cv=3)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
model=grid.best_estimator_